## 4. Data Cleanup and Preparation

__Project - BSTN Capstone__
<br>__Beth McGregor__

### Introduction

This notebook processes data (in csv files) obtained from webscraping the Chicago, Berlin and London Marathon race results.
See the previous notebooks: 
- __1. Web Scraping - Chicago Marathon Results 2014-2019__
- __2. Web Scraping - Berlin Marathon Results 2014-2019__
- __3. Web Scraping - London Marathon Results 2014-2019__

The data from individual years is first combined into a single file for each marathon, and then saved into a csv that is re-loaded for data cleaning and processing. Some races required more processing than others, but overall similar steps were followed: unneccessary columns were removed, missing values were addressed, nationality was separated from the full name column, an elite status column was added, and weather and course information were added. 
<br>
The data for each race was saved into csv files. I created a csv for each race with features common across all 3 marathons. This data will be used in a subsequent notebook for exploratory data analysis and analyzing differences between the races. 
<br>
For the Chicago and London marathons, I also created and saved csv files with the maximum number of features possible. These files will be used in a subsequent notebook for building and running models. 

#### Data Dictionary

This data dictionary describes the features as they will be after processing at the end of this notebook, but I thought it would be helpful to include this information here to provide some context around the various features. 

These features were available for all 3 marathons:
- marathon_name (categorical) 
    - name of marathon
- year
    - year of marathon
- full_name (categorical)
    - first and last name of runner
- elite_status (numeric)
    - identifies whether runner was an elite (1) runner vs. a non-elite runner (0)
    - elite runners must meet fast qualifying standards
- nationality (categorical) 
    - country code (IOC) of nationality of runner
- gender (categorical) 
    - gender (male(0)/female(1)) of runner
- finish_time_seconds (numerical)
    - time in which the runner completes the marathon (42.2 km)
    - converted to seconds for analysis
    - the target variable for this project
- altitude_start (numerical) 
    - altitude at start line of the race
    - units: metres
- altitude_finish (numerical)
    - altitude at the finish line of the race 
    - units: metres
- max_slope (numerical)
    - the maximum percent change in slope within any interval on the marathon course
    - units: percent
- min_slope (numerical)
    - the minimum percent change in slope within any interval on the marathon course
    - units: percent
- elevation gain (numerical)
    - cummulative elevation gain over the marathon course
    - units: metres
- elevation loss (numerical)
    - cummulative elevation loss over the marathon course
    - units: metres
- temperature_min (numerical)
    - minimum temperature recorded on day of the marathon at the closest weather station
    - units °C
- temperature_max (numerical)
    - maximum temperature recorded on day of the marathon at the closest weather station
    - units: °C
- precipitation (numerical)
    - precipitation recorded on day of the marathon at the closest weather station
    - units: mm
- bib_number (categorical)
    - Numeric/alpha-numeric codes are unique per race, but are repeated between races, and race years

These features were not available consistently for all marathons:
- age_class (categorical)
    - age category of runner on day of race
    - available for Chicago and London
- half_split_seconds (numerical)
    - time in which the marathoner completes half the marathon (21.1 km)
    - converted to seconds for analysis
    - available for London and Berlin

#### Combine & Prepare the Chicago Race Results from 2014-2019

To combine each race for addition into the final race dataset, I need to do a bit of processing:
- Combine the individiual races for 2014 to 2019 into one csv file. I sourced some amazing code to help me do this: https://www.freecodecamp.org/news/how-to-combine-multiple-csv-files-with-8-lines-of-code-265183e0854/
- Add a column with the race name to the dataframe. In hindsight, this should have been added during the data scraping phase. 
- Split the nationality from the full_name columns
- Add weather and course features for each marathon
- Create an elite_status column
- Remove unnecessary columns

In [1]:
# Load required libraries
import os
import glob
import pandas as pd

NOTE: The cell below only needed to be run once to retrieve the "combined_chicago.csv". If run multiple times, duplicates are created. 

In [2]:
# Combine the race results for each year into one csv file (source of code is above)
data_dir = 'Data/chicago/'

extension = 'csv'
all_filenames = [i for i in glob.glob(data_dir + '*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
if not os.path.exists(data_dir + 'output'):
    os.makedirs(data_dir + 'output')
combined_csv.to_csv(data_dir + "output/combined_chicago.csv", index=False, encoding='utf-8-sig')

In [3]:
# Load the csv created above
chicago_results_df = pd.read_csv('Data/chicago/output/combined_chicago.csv')
chicago_results_df.head()

,Unnamed: 0,place_overall,place_gender,full_name,bib_number,age_class,finish_time,gender,year
0,0,1,1,"Cherono, Lawrence (KEN)",4,30-34,02:05:45,M,2019
1,1,2,2,"Debela, Dejene (ETH)",38,20-24,02:05:46,M,2019
2,2,3,3,"Mengstu, Asefa (ETH)",5,30-34,02:05:48,M,2019
3,3,4,4,"Karoki, Bedan (KEN)",9,25-29,02:05:53,M,2019
4,4,5,5,"Abdi, Bashir (BEL)",10,30-34,02:06:14,M,2019


In [4]:
# Check for missing values
chicago_results_df.isna().sum()

Unnamed: 0       0
place_overall    0
place_gender     0
full_name        0
bib_number       0
age_class        0
finish_time      0
gender           0
year             0
dtype: int64

In [5]:
# Split the full_name column in to separate nationality, first name and last name columns
chicago_results_df['nationality'] = chicago_results_df['full_name'].str.extract(r'\(([A-Z]{3})\)$')
chicago_results_df['last_name'] = chicago_results_df['full_name'].str.extract(r'^(.+),')
chicago_results_df['first_name'] = chicago_results_df['full_name'].str.extract(r'^.+, (.+) \([A-Z]{3}\)$')


In [6]:

chicago_results_df['first_namex'] = chicago_results_df.loc[chicago_results_df['nationality'].isna(), "full_name"].str.extract(r'^.+, (.+)$')
chicago_results_df['first_name'] = chicago_results_df['first_name'].fillna(chicago_results_df['first_namex'])
chicago_results_df = chicago_results_df.drop('first_namex', axis=1)


In [7]:
chicago_results_df['nationality'] = chicago_results_df['nationality'].fillna('Unknown')

In [8]:
# Check for missing values after the above step
chicago_results_df.isna().sum()

Unnamed: 0       0
place_overall    0
place_gender     0
full_name        0
bib_number       0
age_class        0
finish_time      0
gender           0
year             0
nationality      0
last_name        0
first_name       0
dtype: int64

In [9]:
# Add first_name and last_name columns back together to create full_name
chicago_results_df['full_name']= chicago_results_df['first_name'] + ' ' + chicago_results_df['last_name']
chicago_results_df.head()


,Unnamed: 0,place_overall,place_gender,full_name,bib_number,age_class,finish_time,gender,year,nationality,last_name,first_name
0,0,1,1,Lawrence Cherono,4,30-34,02:05:45,M,2019,KEN,Cherono,Lawrence
1,1,2,2,Dejene Debela,38,20-24,02:05:46,M,2019,ETH,Debela,Dejene
2,2,3,3,Asefa Mengstu,5,30-34,02:05:48,M,2019,ETH,Mengstu,Asefa
3,3,4,4,Bedan Karoki,9,25-29,02:05:53,M,2019,KEN,Karoki,Bedan
4,4,5,5,Bashir Abdi,10,30-34,02:06:14,M,2019,BEL,Abdi,Bashir


#### Add a column with the race name to the dataframe

In [10]:
chicago_results_df['marathon_name'] = 'chicago'

# Check that `race` column was added
chicago_results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253367 entries, 0 to 253366
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     253367 non-null  int64 
 1   place_overall  253367 non-null  int64 
 2   place_gender   253367 non-null  int64 
 3   full_name      253367 non-null  object
 4   bib_number     253367 non-null  int64 
 5   age_class      253367 non-null  object
 6   finish_time    253367 non-null  object
 7   gender         253367 non-null  object
 8   year           253367 non-null  int64 
 9   nationality    253367 non-null  object
 10  last_name      253367 non-null  object
 11  first_name     253367 non-null  object
 12  marathon_name  253367 non-null  object
dtypes: int64(5), object(8)
memory usage: 25.1+ MB


#### Add columns with the course summary data to the dataframe

In [11]:
chicago_results_df['altitude_start'] = 182.94
chicago_results_df['altitude_finish'] = 182.82
chicago_results_df['max_slope'] = 3.80
chicago_results_df['min_slope'] = -5.90
chicago_results_df['elevation_gain'] = 57.91
chicago_results_df['elevation_loss'] = -58.03
chicago_results_df['course_turns'] = 42

In [12]:
# Confirm that these columns were added. 
chicago_results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253367 entries, 0 to 253366
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       253367 non-null  int64  
 1   place_overall    253367 non-null  int64  
 2   place_gender     253367 non-null  int64  
 3   full_name        253367 non-null  object 
 4   bib_number       253367 non-null  int64  
 5   age_class        253367 non-null  object 
 6   finish_time      253367 non-null  object 
 7   gender           253367 non-null  object 
 8   year             253367 non-null  int64  
 9   nationality      253367 non-null  object 
 10  last_name        253367 non-null  object 
 11  first_name       253367 non-null  object 
 12  marathon_name    253367 non-null  object 
 13  altitude_start   253367 non-null  float64
 14  altitude_finish  253367 non-null  float64
 15  max_slope        253367 non-null  float64
 16  min_slope        253367 non-null  floa

#### Add the weather data for each year

In [13]:
# First, create a dictionary with the min and max temperatures and precipitation for each year

chicago_weather_dictionary = {}
chicago_weather_dictionary[2014] = {
    'temperature_min': 7.2,
    'temperature_max': 17.8, 
    'precipitation': 0.00
} 

chicago_weather_dictionary[2015] = {
    'temperature_min': 11.7,
    'temperature_max': 26.7, 
    'precipitation': 0.00
} 


chicago_weather_dictionary[2016] = {
    'temperature_min': 10.0,
    'temperature_max': 17.2, 
    'precipitation': 0.00
} 

chicago_weather_dictionary[2017] = {
    'temperature_min': 13.3,
    'temperature_max': 26.7, 
    'precipitation': 0.00
} 

chicago_weather_dictionary[2018] = {
    'temperature_min': 13.9,
    'temperature_max': 18.3, 
    'precipitation': 20.83
} 

chicago_weather_dictionary[2019] = {
    'temperature_min': 3.3,
    'temperature_max': 13.3, 
    'precipitation': 0.00
} 

# Loop through the dictionary to add the min and max temperatures, and precipitation data for each race year

for y, v in chicago_weather_dictionary.items():
    chicago_results_df.loc[chicago_results_df['year']==y, 'temperature_min']=v['temperature_min']
    chicago_results_df.loc[chicago_results_df['year']==y, 'temperature_max']=v['temperature_max']
    chicago_results_df.loc[chicago_results_df['year']==y, 'precipitation']=v['precipitation']

In [14]:
# Confirm the weather data was added

chicago_results_df.head()

,Unnamed: 0,place_overall,place_gender,full_name,bib_number,age_class,finish_time,gender,year,nationality,...,altitude_start,altitude_finish,max_slope,min_slope,elevation_gain,elevation_loss,course_turns,temperature_min,temperature_max,precipitation
0,0,1,1,Lawrence Cherono,4,30-34,02:05:45,M,2019,KEN,...,182.94,182.82,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0
1,1,2,2,Dejene Debela,38,20-24,02:05:46,M,2019,ETH,...,182.94,182.82,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0
2,2,3,3,Asefa Mengstu,5,30-34,02:05:48,M,2019,ETH,...,182.94,182.82,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0
3,3,4,4,Bedan Karoki,9,25-29,02:05:53,M,2019,KEN,...,182.94,182.82,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0
4,4,5,5,Bashir Abdi,10,30-34,02:06:14,M,2019,BEL,...,182.94,182.82,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0


#### Look at the datatypes

In [15]:
chicago_results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253367 entries, 0 to 253366
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       253367 non-null  int64  
 1   place_overall    253367 non-null  int64  
 2   place_gender     253367 non-null  int64  
 3   full_name        253367 non-null  object 
 4   bib_number       253367 non-null  int64  
 5   age_class        253367 non-null  object 
 6   finish_time      253367 non-null  object 
 7   gender           253367 non-null  object 
 8   year             253367 non-null  int64  
 9   nationality      253367 non-null  object 
 10  last_name        253367 non-null  object 
 11  first_name       253367 non-null  object 
 12  marathon_name    253367 non-null  object 
 13  altitude_start   253367 non-null  float64
 14  altitude_finish  253367 non-null  float64
 15  max_slope        253367 non-null  float64
 16  min_slope        253367 non-null  floa

Data types appear to be appropriate for the data, with the exception of finish_time which will be converted to seconds below. 

#### Update the age categories


In [16]:
# Look at the age categories in the Chicago Marathon results
chicago_results_df['age_class'].unique()

array(['30-34', '20-24', '25-29', '35-39', '40-44', '45-49',
       '19 and under', '50-54', '55-59', '60-64', '70-74', '65-69',
       '<span class="text-muted">–</span>', '75-79', '80+', 'M-15',
       'W-15'], dtype=object)

In [17]:
# Investigate the rows with age_class "<span class="text-muted">–</span>"
chicago_results_df[chicago_results_df['age_class']=='<span class="text-muted">–</span>']

,Unnamed: 0,place_overall,place_gender,full_name,bib_number,age_class,finish_time,gender,year,nationality,...,altitude_start,altitude_finish,max_slope,min_slope,elevation_gain,elevation_loss,course_turns,temperature_min,temperature_max,precipitation
3879,3879,4544,3880,Joshua Durr,20290,"<span class=""text-muted"">–</span>",03:18:01,M,2019,USA,...,182.94,182.82,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0
96889,6473,8668,6474,Donald Cuthbertson,23471,"<span class=""text-muted"">–</span>",03:47:33,M,2016,GBR,...,182.94,182.82,3.8,-5.9,57.91,-58.03,42,10.0,17.2,0.0


In [18]:
# There are 2 rows with "<span class="text-muted">–</span>" - drop these
chicago_results_df = chicago_results_df[chicago_results_df['age_class']!='<span class="text-muted">–</span>']

In [19]:
# Confirm these rows were dropped
chicago_results_df['age_class'].unique()

array(['30-34', '20-24', '25-29', '35-39', '40-44', '45-49',
       '19 and under', '50-54', '55-59', '60-64', '70-74', '65-69',
       '75-79', '80+', 'M-15', 'W-15'], dtype=object)

In [20]:
# Group the runners in age categories < 40 years into an 18-39 category
# This aligns with the London data set
chicago_results_df.loc[chicago_results_df['age_class'].isin(
    ['M-15', 'W-15', '19 and under','20-24', '25-29', '30-34', '35-39']),'age_class'] = '18-39'
                   

In [21]:
# Confirm the age classes were updated
chicago_results_df['age_class'].unique()

array(['18-39', '40-44', '45-49', '50-54', '55-59', '60-64', '70-74',
       '65-69', '75-79', '80+'], dtype=object)

#### Convert finish_time to seconds

In [22]:
def convert_to_seconds(time):
    x = time.split(":")
    return int(x[0])*3600+int(x[1])*60+int(x[2])

chicago_results_df['finish_time_seconds'] = chicago_results_df['finish_time'].apply(convert_to_seconds)

#### Create an elite_status column

In [23]:
# Remove extra characters from elite runner Kohei Matsumura's full_name
chicago_results_df.loc[chicago_results_df['full_name']=='Kohei Matsumura\xa0','full_name']='Kohei Matsumura'

In [24]:
# Create a dictionary of lists of elite runners for every year in the data set. 

elite_names = {
    2014: ['Kenenisa Bekele', 'Eliud Kipchoge','Bernard Koech', 'Sammy Kitwara',  'Dickson Chumba', 'Wesley Korir', 'Koji Kobayashi', 'Satoru Sasaki', 'Ryosuke Fukuyama', 'Nobuaki Takata', 'Rui Yonezawa', 'Naoki Okamoto', 'Bobby Curtis', 'Patrick Rizzo', 'Craig Leon', 'Carlos Trujillo', 'Mike Morgan', 'Luke Humphrey', 'Christo Landry', 'Mario Macias', 'Jameson Mora', 'Tim Young', 'Gabe Proctor', 'Jesse Davis', 'Dan Kremske', 'Jake Riley', 'Matthew Llano', 'Brendan Gregg', 'Ryan Kienzle', 'Leonardo Tenepaguay', 'Bayron Piedra Aviles', 'Florence Kiplagat', 'Mare Dibaba', 'Birhane Dibaba', 'Amy Hastings', 'Clara Santucci', 'Lisa Uhl', 'Lindsey Scherf', 'Melissa White', 'Wendy Thomas', 'Lauren Jimison', 'Laura Portis', 'Sarah Crouch', 'Sarah Cummings','Heidi Greenwood', 'Whitney Bevins-Lazzara', 'Tina Muir'], 
    2015: ['Sammy Kitwara', 'Wesley Korir', 'Fernando Cabada', 'Luke Puskedra', 'Brandon Mull', 'Deena Kastor', 'Sara Hall', 'Sarah Crouch', 'Diane Nukuri', 'Florence Kiplagat', 'Fionnuala Britton', 'Abera Kuma', 'Mulu Seboka', 'Birhane Dibaba', 'Yebrgual Melese', 'Meskerem Assefa', 'Dickson Chumba', 'Sammy Ndungu', 'Lucas Rotich', 'Tera Moody', 'Taylor Ward', 'Dawn Grunnagle', 'Malcolm Richards', 'Scott Macpherson', 'Chris Erichsen', 'Scott Wietecha', 'Ethan Shaw', 'Tony Migliozzi', 'Christopher Pannone', 'Dan Kremske', 'Christian Thompson', 'Elkanah Kibet', 'Mohamed Hrezi', 'Liam Adams', 'Rob Watson', 'Kayoko Fukushi', 'Susan Partridge', 'Jessica Draskau Petersson', 'Monika Juodeskaite', 'Satoshi Yoshii'], 
    2016: ['Ryoichi Matsuo', 'Takuya Fukatsu', 'Diego Estrada', 'Stephen Sambu', 'Kazuya Ishida', 'Valentine Kipketer', 'Purity Rionoripo', 'Alia Gray', 'Abel Kirui', 'Florence Kiplagat', 'Yebrgual Melese', 'Atsede Baysa', 'Paul Lonyangata', 'Sarah Crouch', 'Dickson Chumba', 'Abayneh Ayele', 'Elkanah Kibet', 'Tim Young', 'Gideon Kipketer', 'Serena Burla', 'Edna Kiplagat', 'Luke Puskedra', 'Koji Gokaya', 'Agnieszka Mierzejewska', 'Micah Kogo', 'Freya Ross'],
    2017: ['Ryoichi Matsuo', 'Dot McMahan', 'Diego Estrada', 'Jessica Draskau Petersson', 'Stephen Sambu', 'Ezekiel Chebii', 'Lisa Weightman', 'Maegan Krifchin', 'Aaron Braun', 'Danna Herrick', 'Valentine Kipketer', 'Madai Perez', 'Alia Gray', 'Becky Wade', 'Abel Kirui', 'Brigid Kosgei', 'Sam Chelanga', 'Sisay Lemma', 'Galen Rupp', 'Feyisa Lilesa', 'Zersenay Tadese', 'Kohei Matsumura', 'Bernard Kipyego', 'Chris Derrick', 'Andrew Bumbalough', 'Noah Droddy', 'Sarah Crouch', 'Chihiro Miyawaki', 'Jordan Hasay', 'Luke Puskedra', 'Tirunesh Dibaba', 'Luke Humphrey'],
    2018: ['Chirine Njeim', 'Geoffrey Kirui', 'Alexi Pappas', 'Aaron Braun', 'Andrew Epperson', 'Patrick Rizzo', 'Gwen Jorgensen', 'Bedan Karoki', 'Birhanu Legese', 'Melanie Myrand', 'Kenneth Kipkemoi', 'Abel Kirui', 'Bayron Piedra', 'Jonas Hampton', 'Brigid Kosgei', 'Florence Kiplagat', 'Alan Peterson', 'Galen Rupp', 'Tyler McCandless', 'Taylor Ward', 'Yohei Suzuki', 'Katie Matthews', 'Yuki Kawauchi', 'Parker Stinson', 'Bernard Kipyego', 'Roza Dereje', 'Hugh Williams', 'Taku Fujimoto', 'Johnny Crain', 'Sarah Crouch', 'Pardon Ndhlovu', 'Elkanah Kibet', 'Ryan Root', 'Kiya Dandena', 'Suguru Osako', 'Mo Farah', 'Shure Demise', 'George Alex', 'Jess Draskau Petersson', 'Mosinet Geremew Bayih', 'Mohamed Reda El Aaraby'],
    2019: ['Asefa Mengstu', 'Brendan Gregg', 'Minato Oishi', 'Diego Estrada', 'Scott Smith', 'Jacob Riley', 'Lisa Weightman', 'Seifu Tura', 'Bedan Karoki', 'Sarah Sellers', 'Madai Perez', 'Brian Shrader', 'Brigid Kosgei', 'Lindsay Flanagan', 'Taylor Ward', 'Ababel Yeshaneh', 'Parker Stinson', 'Tsubasa Hayakawa', 'Bashir Abdi', 'Stephanie Bruce', 'Andrew Bumbalough', 'Noah Droddy', 'Lawrence Cherono', 'Gelete Burka', 'Ryoma Takeuchi', 'Dickson Chumba', 'Laura Thweatt', 'Emma Bates', 'Mo Farah', 'Anke Esser', 'Dejene Debela', 'Fionnuala Mccormack']
}
elite_names

# Create elite_status column and set default value to 0 (non-elite)
chicago_results_df['elite_status'] = 0

# Loop through names in elite_names dictionary above and add 1 in elite_status column if name is found in dictionary
for k, v in elite_names.items():
    print(k) # Print year
    print('Expected Length:', len(v)) # Check how many names in dictionary list
    print('Actual Length:', len(chicago_results_df.loc[(chicago_results_df['year']==k) & (chicago_results_df['place_overall']<400) & (chicago_results_df['full_name'].isin(v))])) # Check how many names found in dataset of those in dictionary list
    
    chicago_results_df.loc[(chicago_results_df['year']==k) & (chicago_results_df['place_overall']<400) & (chicago_results_df['full_name'].isin(v)), 'elite_status'] = 1
                                            


2014
Expected Length: 47
Actual Length: 47
2015
Expected Length: 40
Actual Length: 40
2016
Expected Length: 26
Actual Length: 26
2017
Expected Length: 32
Actual Length: 32
2018
Expected Length: 41
Actual Length: 41
2019
Expected Length: 32
Actual Length: 32


The Chicago Marathon does not have an official 'elite' runner results list posted after the race. I used press releases published prior to the race to build the elite runner lists for each year in the dictionary. Not all runners in the press releases ended up competing in the race so there was some discrepancy between the list created and runners present in the dataset. The code below was used as a check to loop through the names and try and determine differences more efficiently. The end result was an updated list of runners for each year that I could copy and re-paste into the dictionary above. 

In [25]:
# Check/validation of the elite runner list
year = 2019
x = chicago_results_df.loc[
    (chicago_results_df['year']==year) & 
    (chicago_results_df['place_overall']<400) & 
    (chicago_results_df['full_name'].isin(elite_names[year]))
]['full_name'].values
missing_names = list(set(elite_names[year]).difference(set(x)))

to_delete = []
to_add = []
for n in missing_names:
    name = n.capitalize()
    y = chicago_results_df.loc[
        (chicago_results_df['year']==year) & 
        (chicago_results_df['place_overall']<400) & 
        (chicago_results_df['full_name'].str.contains(name, case=False))
    ]
    if(len(y) > 0):
        print("%s should be %s" % (n, y['full_name'].values[0]))
        to_delete.append(n)
        to_add.append(y['full_name'].values[0])
    else:
        to_delete.append(n)

# for n in missing_names:
#     name = n.split(" ")[1].capitalize()
#     y = chicago_results_df.loc[
#         (chicago_results_df['year']==year) & 
#         (chicago_results_df['place_overall']<400) & 
#         (chicago_results_df['full_name'].str.contains(name))
#     ]
#     if(len(y) > 0):
#         print("%s should be %s" % (n, y['full_name'].values[0]))
#         to_delete.append(n)
#         to_add.append(y['full_name'].values[0])
#     else:
#         to_delete.append(n)
print("%s deleted" % len(to_delete))
print("Replace: ", list(set(elite_names[year]).difference(set(to_delete)))+to_add)

0 deleted
Replace:  ['Brendan Gregg', 'Brigid Kosgei', 'Gelete Burka', 'Brian Shrader', 'Seifu Tura', 'Mo Farah', 'Ababel Yeshaneh', 'Sarah Sellers', 'Tsubasa Hayakawa', 'Madai Perez', 'Lawrence Cherono', 'Emma Bates', 'Lindsay Flanagan', 'Fionnuala Mccormack', 'Jacob Riley', 'Taylor Ward', 'Dickson Chumba', 'Stephanie Bruce', 'Asefa Mengstu', 'Parker Stinson', 'Andrew Bumbalough', 'Scott Smith', 'Ryoma Takeuchi', 'Laura Thweatt', 'Dejene Debela', 'Bashir Abdi', 'Bedan Karoki', 'Lisa Weightman', 'Anke Esser', 'Noah Droddy', 'Minato Oishi', 'Diego Estrada']


In [26]:
# This code was used to investigate differences in the spelling of names. 
year = 2019
name = "Cherono"
chicago_results_df.loc[
    (chicago_results_df['year']==year) & 
    (chicago_results_df['place_overall']<400) & 
    (chicago_results_df['full_name'].str.contains(name, case=False))
]

,Unnamed: 0,place_overall,place_gender,full_name,bib_number,age_class,finish_time,gender,year,nationality,...,max_slope,min_slope,elevation_gain,elevation_loss,course_turns,temperature_min,temperature_max,precipitation,finish_time_seconds,elite_status
0,0,1,1,Lawrence Cherono,4,18-39,02:05:45,M,2019,KEN,...,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0,7545,1


#### Drop columns that are no longer needed


In [27]:
chicago_results_df = chicago_results_df.drop(['Unnamed: 0', 'place_gender', 'last_name', 'first_name', 'finish_time'], axis = 1)
chicago_results_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253365 entries, 0 to 253366
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   place_overall        253365 non-null  int64  
 1   full_name            253365 non-null  object 
 2   bib_number           253365 non-null  int64  
 3   age_class            253365 non-null  object 
 4   gender               253365 non-null  object 
 5   year                 253365 non-null  int64  
 6   nationality          253365 non-null  object 
 7   marathon_name        253365 non-null  object 
 8   altitude_start       253365 non-null  float64
 9   altitude_finish      253365 non-null  float64
 10  max_slope            253365 non-null  float64
 11  min_slope            253365 non-null  float64
 12  elevation_gain       253365 non-null  float64
 13  elevation_loss       253365 non-null  float64
 14  course_turns         253365 non-null  int64  
 15  temperature_min  

In [28]:
# Check for duplicate rows
chicago_results_df.duplicated().sum()

0

There are no duplicated rows in the Chicago Marathon results.

In [29]:
# Create a dataframe wiht a reduced subset of results that matches the Berlin and London Marathons

chicago_results_reduced_df = chicago_results_df[['marathon_name', 'year', 'place_overall', 'full_name', 'elite_status', 'nationality', 'gender', 'finish_time_seconds', 'altitude_start', 'altitude_finish', 'max_slope', 'min_slope', 'elevation_gain', 'elevation_loss', 'course_turns', 'temperature_min', 'temperature_max', 'precipitation']]
chicago_results_reduced_df.head()


,marathon_name,year,place_overall,full_name,elite_status,nationality,gender,finish_time_seconds,altitude_start,altitude_finish,max_slope,min_slope,elevation_gain,elevation_loss,course_turns,temperature_min,temperature_max,precipitation
0,chicago,2019,1,Lawrence Cherono,1,KEN,M,7545,182.94,182.82,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0
1,chicago,2019,2,Dejene Debela,1,ETH,M,7546,182.94,182.82,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0
2,chicago,2019,3,Asefa Mengstu,1,ETH,M,7548,182.94,182.82,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0
3,chicago,2019,4,Bedan Karoki,1,KEN,M,7553,182.94,182.82,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0
4,chicago,2019,5,Bashir Abdi,1,BEL,M,7574,182.94,182.82,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0


In [30]:
# Check the shape of the chicago results

chicago_results_reduced_df.shape

(253365, 18)

In [31]:
# One final check for missing values

chicago_results_reduced_df.isna().sum()

marathon_name          0
year                   0
place_overall          0
full_name              0
elite_status           0
nationality            0
gender                 0
finish_time_seconds    0
altitude_start         0
altitude_finish        0
max_slope              0
min_slope              0
elevation_gain         0
elevation_loss         0
course_turns           0
temperature_min        0
temperature_max        0
precipitation          0
dtype: int64

In [32]:
# Save the chicago_results_df to a csv

chicago_results_reduced_df.to_csv('Data/chicago/output/chicago_results_reduced_df.csv')

In [33]:
# Create a dataframe with as many features as possible for Chicago
# Create a dataframe retaining as many features as possible for London
chicago_results_all_features_df = chicago_results_df[['marathon_name', 'year', 'place_overall', 'full_name', 'bib_number', 'age_class', 'elite_status', 'nationality', 'gender', 'finish_time_seconds', 'altitude_start', 'altitude_finish', 'max_slope', 'min_slope', 'elevation_gain', 'elevation_loss', 'course_turns', 'temperature_min', 'temperature_max', 'precipitation']]
chicago_results_all_features_df.head()

,marathon_name,year,place_overall,full_name,bib_number,age_class,elite_status,nationality,gender,finish_time_seconds,altitude_start,altitude_finish,max_slope,min_slope,elevation_gain,elevation_loss,course_turns,temperature_min,temperature_max,precipitation
0,chicago,2019,1,Lawrence Cherono,4,18-39,1,KEN,M,7545,182.94,182.82,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0
1,chicago,2019,2,Dejene Debela,38,18-39,1,ETH,M,7546,182.94,182.82,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0
2,chicago,2019,3,Asefa Mengstu,5,18-39,1,ETH,M,7548,182.94,182.82,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0
3,chicago,2019,4,Bedan Karoki,9,18-39,1,KEN,M,7553,182.94,182.82,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0
4,chicago,2019,5,Bashir Abdi,10,18-39,1,BEL,M,7574,182.94,182.82,3.8,-5.9,57.91,-58.03,42,3.3,13.3,0.0


In [34]:
# Check the shape of the all features dataframe

chicago_results_all_features_df.shape

(253365, 20)

In [35]:
# Save chicago_results_all_features_df to a csv
chicago_results_all_features_df.to_csv("Data/chicago/output/chicago_results_all_features.csv")

### Combine & Prepare the Berlin Race Results from 2014-2019

NOTE: The code only needs to be run once to create the "combined_berlin.csv"

In [36]:
data_dir = 'Data/berlin/'

extension = 'csv'
all_filenames = [i for i in glob.glob(data_dir + '*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
if not os.path.exists(data_dir + 'output'):
    os.makedirs(data_dir + 'output')
combined_csv.to_csv(data_dir + "output/combined_berlin.csv", index=False, encoding='utf-8-sig')

In [37]:
berlin_results_df = pd.read_csv('Data/berlin/output/combined_berlin.csv')
berlin_results_df.head()

,Unnamed: 0,place_overall,full_name,nationality,bib_number,half_split,finish_time,gender,year
0,0,1,Eliud Kipchoge,KEN,4,01:01:53,02:04:00,M,2015
1,1,2,Eliud Kiptanui,KEN,6,01:01:53,02:05:21,M,2015
2,2,3,Feyisa Lilesa,ETH,5,01:01:53,02:06:57,M,2015
3,3,4,Emmanuel Mutai,KEN,3,01:01:53,02:07:46,M,2015
4,4,5,Geoffrey Mutai,KEN,2,01:01:53,02:09:29,M,2015


In [38]:
# Add a column with the race name to the dataframe

berlin_results_df['marathon_name'] = 'berlin'

# Check that `race` column was added
berlin_results_df.head()
berlin_results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225533 entries, 0 to 225532
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     225533 non-null  int64 
 1   place_overall  225533 non-null  int64 
 2   full_name      225533 non-null  object
 3   nationality    225531 non-null  object
 4   bib_number     225533 non-null  object
 5   half_split     225486 non-null  object
 6   finish_time    225533 non-null  object
 7   gender         225533 non-null  object
 8   year           225533 non-null  int64 
 9   marathon_name  225533 non-null  object
dtypes: int64(3), object(7)
memory usage: 17.2+ MB


#### Add columns with the course summary data to the dataframe

In [39]:
berlin_results_df['altitude_start'] = 33.99
berlin_results_df['altitude_finish'] = 33.99
berlin_results_df['max_slope'] = 1.60
berlin_results_df['min_slope'] = -1.70
berlin_results_df['elevation_gain'] = 77.11
berlin_results_df['elevation_loss'] = -77.11
berlin_results_df['course_turns'] = 45

#### Add the weather data for each year

In [40]:
# First, create a dictionary with the min and max temperatures and precipitation for each year

berlin_weather_dictionary = {}
berlin_weather_dictionary[2014] = {
    'temperature_min': 5.8,
    'temperature_max': 21.5, 
    'precipitation': 0.00
} 

berlin_weather_dictionary[2015] = {
    'temperature_min': 6.5,
    'temperature_max': 17.4, 
    'precipitation': 0.00
} 


berlin_weather_dictionary[2016] = {
    'temperature_min': 8.8,
    'temperature_max': 22.8, 
    'precipitation': 0.00
} 

berlin_weather_dictionary[2017] = {
    'temperature_min': 12.7,
    'temperature_max': 21.1, 
    'precipitation': 1.40
} 

berlin_weather_dictionary[2018] = {
    'temperature_min': 10.1,
    'temperature_max': 23.2, 
    'precipitation': 0.00
} 

berlin_weather_dictionary[2019] = {
    'temperature_min': 12.8,
    'temperature_max': 17.0, 
    'precipitation': 8.00
} 

# Loop through the dictionary to add the min and max temperatures, and precipitation data for each race year

for y, v in berlin_weather_dictionary.items():
    berlin_results_df.loc[berlin_results_df['year']==y, 'temperature_min']=v['temperature_min']
    berlin_results_df.loc[berlin_results_df['year']==y, 'temperature_max']=v['temperature_max']
    berlin_results_df.loc[berlin_results_df['year']==y, 'precipitation']=v['precipitation']

In [41]:
# Confirm the weather data was added

berlin_results_df.head()

,Unnamed: 0,place_overall,full_name,nationality,bib_number,half_split,finish_time,gender,year,marathon_name,altitude_start,altitude_finish,max_slope,min_slope,elevation_gain,elevation_loss,course_turns,temperature_min,temperature_max,precipitation
0,0,1,Eliud Kipchoge,KEN,4,01:01:53,02:04:00,M,2015,berlin,33.99,33.99,1.6,-1.7,77.11,-77.11,45,6.5,17.4,0.0
1,1,2,Eliud Kiptanui,KEN,6,01:01:53,02:05:21,M,2015,berlin,33.99,33.99,1.6,-1.7,77.11,-77.11,45,6.5,17.4,0.0
2,2,3,Feyisa Lilesa,ETH,5,01:01:53,02:06:57,M,2015,berlin,33.99,33.99,1.6,-1.7,77.11,-77.11,45,6.5,17.4,0.0
3,3,4,Emmanuel Mutai,KEN,3,01:01:53,02:07:46,M,2015,berlin,33.99,33.99,1.6,-1.7,77.11,-77.11,45,6.5,17.4,0.0
4,4,5,Geoffrey Mutai,KEN,2,01:01:53,02:09:29,M,2015,berlin,33.99,33.99,1.6,-1.7,77.11,-77.11,45,6.5,17.4,0.0


#### Look at the datatypes

In [42]:
berlin_results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225533 entries, 0 to 225532
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       225533 non-null  int64  
 1   place_overall    225533 non-null  int64  
 2   full_name        225533 non-null  object 
 3   nationality      225531 non-null  object 
 4   bib_number       225533 non-null  object 
 5   half_split       225486 non-null  object 
 6   finish_time      225533 non-null  object 
 7   gender           225533 non-null  object 
 8   year             225533 non-null  int64  
 9   marathon_name    225533 non-null  object 
 10  altitude_start   225533 non-null  float64
 11  altitude_finish  225533 non-null  float64
 12  max_slope        225533 non-null  float64
 13  min_slope        225533 non-null  float64
 14  elevation_gain   225533 non-null  float64
 15  elevation_loss   225533 non-null  float64
 16  course_turns     225533 non-null  int6

#### Convert columns that don't match other race dataframes

In [43]:
# Determine if there are any DSQ results and remove
berlin_results_df = berlin_results_df[berlin_results_df['finish_time']!='DSQ']

In [44]:
# Get rid of results with this error text
berlin_results_df = berlin_results_df[berlin_results_df['finish_time'] != 'An error occured, please try again in a few minutes.']

In [45]:
# Convert finish_time to seconds

def convert_to_seconds(time):
    x = time.split(":")
    return int(x[0])*3600+int(x[1])*60+int(x[2])

berlin_results_df['finish_time_seconds'] = berlin_results_df['finish_time'].apply(convert_to_seconds)

#### Create an elite status column
Unfortunately, I was unable to find any official record of the elite field for past berlin marathons. For the interim, I decided to set a time cutoff for men (2:15:00) and women (2:50:00) for each year and designate them as "elite". With additional time and research, this information could potentially be pulled together. 

In [46]:
# Create elite_status column and set default value to 0 (non-elite)
berlin_results_df['elite_status'] = 0

# Set finish time cutoffs to set elite status
elite_time_cutoff_seconds_men = 2*3600+15*60
elite_time_cutoff_seconds_women = 2*3600+50*60

# Add elite status to elite_status column
berlin_results_df.loc[(berlin_results_df['finish_time_seconds'] <= elite_time_cutoff_seconds_men) 
                      & (berlin_results_df['gender'] == 'M'), 'elite_status'] = 1
berlin_results_df.loc[(berlin_results_df['finish_time_seconds'] <= elite_time_cutoff_seconds_women) 
                      & (berlin_results_df['gender'] == 'W'), 'elite_status'] = 1

#### Check for missing values

In [47]:
berlin_results_df.isna().sum()

Unnamed: 0              0
place_overall           0
full_name               0
nationality             2
bib_number              0
half_split             47
finish_time             0
gender                  0
year                    0
marathon_name           0
altitude_start          0
altitude_finish         0
max_slope               0
min_slope               0
elevation_gain          0
elevation_loss          0
course_turns            0
temperature_min         0
temperature_max         0
precipitation           0
finish_time_seconds     0
elite_status            0
dtype: int64

There are 2 missing nationality values and 47 missing half_split values. The missing half_split rows will likely be dropped, but will be investigated further below before doing so. I will attempt to fill in the 2 missing nationality values. 

In [48]:
berlin_results_df[berlin_results_df['nationality'].isna()]

,Unnamed: 0,place_overall,full_name,nationality,bib_number,half_split,finish_time,gender,year,marathon_name,...,max_slope,min_slope,elevation_gain,elevation_loss,course_turns,temperature_min,temperature_max,precipitation,finish_time_seconds,elite_status
41189,436,4437,Dirk Schï¿½lkens,NaN,26000,01:40:27,03:24:50,M,2014,berlin,...,1.6,-1.7,77.11,-77.11,45,5.8,21.5,0.0,12290,0
219049,157,24887,Wolfram Barris,NaN,4239,02:16:27,05:10:42,M,2018,berlin,...,1.6,-1.7,77.11,-77.11,45,10.1,23.2,0.0,18642,0


In [49]:
# Replace NaN values with 'Unknown'
# This will match the Chicago data

berlin_results_df['nationality'] = berlin_results_df['nationality'].fillna('Unknown')


In [50]:
# Look at the missing half_split values
berlin_results_df[berlin_results_df['half_split'].isna()]

,Unnamed: 0,place_overall,full_name,nationality,bib_number,half_split,finish_time,gender,year,marathon_name,...,max_slope,min_slope,elevation_gain,elevation_loss,course_turns,temperature_min,temperature_max,precipitation,finish_time_seconds,elite_status
144502,586,3349,Erik Haugvaldstad,NOR,57805,NaN,03:08:10,M,2019,berlin,...,1.6,-1.7,77.11,-77.11,45,12.8,17.0,8.0,11290,0
146801,885,5398,Peter Yeates,IRL,42165,NaN,03:19:48,M,2019,berlin,...,1.6,-1.7,77.11,-77.11,45,12.8,17.0,8.0,11988,0
147346,430,5876,Adam Kearns,USA,24838,NaN,03:22:14,M,2019,berlin,...,1.6,-1.7,77.11,-77.11,45,12.8,17.0,8.0,12134,0
148716,800,750,Nathalya Gobbo,BRA,20450,NaN,03:27:05,W,2019,berlin,...,1.6,-1.7,77.11,-77.11,45,12.8,17.0,8.0,12425,0
149769,853,913,Viviane Jaekel,GER,23772,NaN,03:29:50,W,2019,berlin,...,1.6,-1.7,77.11,-77.11,45,12.8,17.0,8.0,12590,0
150774,858,8755,Jeremy Poullin,FRA,33173,NaN,03:33:35,M,2019,berlin,...,1.6,-1.7,77.11,-77.11,45,12.8,17.0,8.0,12815,0
151331,415,1212,Angela Suarez,COL,69262,NaN,03:35:37,W,2019,berlin,...,1.6,-1.7,77.11,-77.11,45,12.8,17.0,8.0,12937,0
151430,514,9283,Cedric Kante,FRA,24624,NaN,03:35:57,M,2019,berlin,...,1.6,-1.7,77.11,-77.11,45,12.8,17.0,8.0,12957,0
152419,503,1437,Anita Kempf,SUI,59761,NaN,03:38:58,W,2019,berlin,...,1.6,-1.7,77.11,-77.11,45,12.8,17.0,8.0,13138,0
152866,950,10414,Daniel Edsenius,SWE,18156,NaN,03:40:09,M,2019,berlin,...,1.6,-1.7,77.11,-77.11,45,12.8,17.0,8.0,13209,0


The missing half split values are all from 2019. Some of the runners have very close finish times, suggesting that there could have been an issue with the timing mat recording the half_split times. The 47 rows missing half_split times will be dropped. 

In [51]:
# Drop the missing age_class values (NaN)
berlin_results_df = berlin_results_df[~berlin_results_df['half_split'].isna()]

In [52]:
#Re-check missing values
berlin_results_df.isna().sum()

Unnamed: 0             0
place_overall          0
full_name              0
nationality            0
bib_number             0
half_split             0
finish_time            0
gender                 0
year                   0
marathon_name          0
altitude_start         0
altitude_finish        0
max_slope              0
min_slope              0
elevation_gain         0
elevation_loss         0
course_turns           0
temperature_min        0
temperature_max        0
precipitation          0
finish_time_seconds    0
elite_status           0
dtype: int64

In [53]:
# Convert half_split time to seconds

def convert_to_seconds(time):
    x = time.split(":")
    return int(x[0])*3600+int(x[1])*60+int(x[2])

berlin_results_df['half_split_seconds'] = berlin_results_df['half_split'].apply(convert_to_seconds)

In [54]:
# Check for duplicated rows
berlin_results_df.duplicated().sum()

0

There are no duplicated rows in the Berlin Marathon results. 

#### Drop columns that are no longer needed/unnecessary

In [55]:
berlin_results_reduced_df = berlin_results_df.drop(['Unnamed: 0', 'half_split', 'finish_time'], axis = 1)
berlin_results_reduced_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225486 entries, 0 to 225532
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   place_overall        225486 non-null  int64  
 1   full_name            225486 non-null  object 
 2   nationality          225486 non-null  object 
 3   bib_number           225486 non-null  object 
 4   gender               225486 non-null  object 
 5   year                 225486 non-null  int64  
 6   marathon_name        225486 non-null  object 
 7   altitude_start       225486 non-null  float64
 8   altitude_finish      225486 non-null  float64
 9   max_slope            225486 non-null  float64
 10  min_slope            225486 non-null  float64
 11  elevation_gain       225486 non-null  float64
 12  elevation_loss       225486 non-null  float64
 13  course_turns         225486 non-null  int64  
 14  temperature_min      225486 non-null  float64
 15  temperature_max  

#### Create 2 csv files, one with reduced features that match the Chicago and London datasets and one with all features


In [56]:
# Select & Reorder columns for all features
berlin_results_all_features_df = berlin_results_df[['marathon_name', 'year', 'place_overall', 'full_name', 'bib_number', 'elite_status', 'nationality', 'gender', 'half_split_seconds', 'finish_time_seconds', 'altitude_start', 'altitude_finish', 'max_slope', 'min_slope', 'elevation_gain', 'elevation_loss', 'course_turns', 'temperature_min', 'temperature_max', 'precipitation']]
berlin_results_all_features_df.head()

,marathon_name,year,place_overall,full_name,bib_number,elite_status,nationality,gender,half_split_seconds,finish_time_seconds,altitude_start,altitude_finish,max_slope,min_slope,elevation_gain,elevation_loss,course_turns,temperature_min,temperature_max,precipitation
0,berlin,2015,1,Eliud Kipchoge,4,1,KEN,M,3713,7440,33.99,33.99,1.6,-1.7,77.11,-77.11,45,6.5,17.4,0.0
1,berlin,2015,2,Eliud Kiptanui,6,1,KEN,M,3713,7521,33.99,33.99,1.6,-1.7,77.11,-77.11,45,6.5,17.4,0.0
2,berlin,2015,3,Feyisa Lilesa,5,1,ETH,M,3713,7617,33.99,33.99,1.6,-1.7,77.11,-77.11,45,6.5,17.4,0.0
3,berlin,2015,4,Emmanuel Mutai,3,1,KEN,M,3713,7666,33.99,33.99,1.6,-1.7,77.11,-77.11,45,6.5,17.4,0.0
4,berlin,2015,5,Geoffrey Mutai,2,1,KEN,M,3713,7769,33.99,33.99,1.6,-1.7,77.11,-77.11,45,6.5,17.4,0.0


In [57]:
# Check the shape of the all features dataframe
berlin_results_all_features_df.shape

(225486, 20)

In [58]:
# Do a final check for missing values
berlin_results_all_features_df.isna().sum()

marathon_name          0
year                   0
place_overall          0
full_name              0
bib_number             0
elite_status           0
nationality            0
gender                 0
half_split_seconds     0
finish_time_seconds    0
altitude_start         0
altitude_finish        0
max_slope              0
min_slope              0
elevation_gain         0
elevation_loss         0
course_turns           0
temperature_min        0
temperature_max        0
precipitation          0
dtype: int64

In [107]:
# Save this dataframe to a csv
berlin_results_all_features_df.to_csv('Data/berlin/output/berlin_results_all_features.csv')

#### Select & re-order columns for reduced data set

In [60]:
# Reduced data set
berlin_results_reduced_df = berlin_results_df[['marathon_name', 'year', 'place_overall', 'full_name', 'elite_status', 'nationality', 'gender', 'finish_time_seconds', 'altitude_start', 'altitude_finish', 'max_slope', 'min_slope', 'elevation_gain', 'elevation_loss', 'course_turns', 'temperature_min', 'temperature_max', 'precipitation']]
berlin_results_reduced_df.head()

,marathon_name,year,place_overall,full_name,elite_status,nationality,gender,finish_time_seconds,altitude_start,altitude_finish,max_slope,min_slope,elevation_gain,elevation_loss,course_turns,temperature_min,temperature_max,precipitation
0,berlin,2015,1,Eliud Kipchoge,1,KEN,M,7440,33.99,33.99,1.6,-1.7,77.11,-77.11,45,6.5,17.4,0.0
1,berlin,2015,2,Eliud Kiptanui,1,KEN,M,7521,33.99,33.99,1.6,-1.7,77.11,-77.11,45,6.5,17.4,0.0
2,berlin,2015,3,Feyisa Lilesa,1,ETH,M,7617,33.99,33.99,1.6,-1.7,77.11,-77.11,45,6.5,17.4,0.0
3,berlin,2015,4,Emmanuel Mutai,1,KEN,M,7666,33.99,33.99,1.6,-1.7,77.11,-77.11,45,6.5,17.4,0.0
4,berlin,2015,5,Geoffrey Mutai,1,KEN,M,7769,33.99,33.99,1.6,-1.7,77.11,-77.11,45,6.5,17.4,0.0


In [61]:
berlin_results_reduced_df.shape

(225486, 18)

In [106]:
# Save this dataframe to a csv
berlin_results_reduced_df.to_csv('Data/berlin/output/berlin_results_reduced.csv')

### Combine & Prepare the London Race Results from 2014-2019

NOTE: The London results have different 'place_overall' for elite runners vs. other runners so there are multiple 'place_overall' for each year for London. The bib_numbers are also treated as a separate series for elite vs. non-elite athletes. An elite_status column will be created below to distinguish official elite athletes. 

The London data required more pre-processing before the results from each year could be combined. The results csv for each year was loaded separately to fix column names in order for consistency across the years. Once the results were consistent they were combined into a single csv file and processed similar to the Chicago and Berlin results. 

<br>NOTE: The pre-processing and combining steps only need to be run once to generate the combined csv file. If run multiple times, duplicates will be created. 

In [63]:
# Load the data to fix issues with column names before joining dataframes together
london_2014_df = pd.read_csv('Data/london/London_race_results_2014.csv')
london_2015_df = pd.read_csv('Data/london/London_race_results_2015.csv')
london_2016_df = pd.read_csv('Data/london/London_race_results_2016.csv')
london_2017_df = pd.read_csv('Data/london/London_race_results_2017.csv')
london_2018_df = pd.read_csv('Data/london/London_race_results_2018.csv')

london_2014_df.info()

/Users/bethmcgregor/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35912 entries, 0 to 35911
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              35912 non-null  int64 
 1   Unnamed: 0.1            35912 non-null  int64 
 2   Unnamed: 0.1.1          35912 non-null  int64 
 3   Unnamed: 0.1.1.1        35912 non-null  int64 
 4   Unnamed: 0.1.1.1.1      35912 non-null  int64 
 5   Unnamed: 0.1.1.1.1.1    35912 non-null  int64 
 6   Unnamed: 0.1.1.1.1.1.1  35912 non-null  int64 
 7   place_overall           35912 non-null  object
 8   place_gender            35912 non-null  object
 9   full_name               35912 non-null  object
 10  bib_number              35912 non-null  int64 
 11  age_class               35906 non-null  object
 12  half_split              35824 non-null  object
 13  finish_time             35912 non-null  object
 14  gender                  35912 non-null  object
 15  ye

/Users/bethmcgregor/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7,8,10,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [64]:
# Fix data scraping screw ups and rename columns
#2014
london_2014_df.rename(columns = {
                                'Place overall': 'place_overall', 
                                'Place gender': 'place_gender', 
                                'Name': 'full_name', 
                                'Runner no':'bib_number',
                                'Category': 'age_class', 
                                'HALF': 'half_split', 
                                'Finish time':'finish_time', 
                                }, inplace = True)

london_2014_df.to_csv( "Data/london/London_race_results_2014.csv")

#2015
london_2015_df.rename(columns = {
                                'Place overall': 'place_overall', 
                                'Place gender': 'place_gender', 
                                'Name': 'full_name', 
                                'Runner no':'bib_number',
                                'Category': 'age_class', 
                                'HALF': 'half_split', 
                                'FINISH':'finish_time', 
                                }, inplace = True)

london_2015_df.to_csv( "Data/london/London_race_results_2015.csv")

#2016
london_2016_df.rename(columns = {
                                'Place overall': 'place_overall', 
                                'Place gender': 'place_gender', 
                                'Name': 'full_name', 
                                'Runner no':'bib_number',
                                'Category': 'age_class', 
                                'HALF': 'half_split', 
                                'FINISH':'finish_time', 
                                }, inplace = True)

london_2016_df.to_csv( "Data/london/London_race_results_2016.csv")

#2017
london_2017_df.rename(columns = {
                                'Place overall': 'place_overall', 
                                'Place gender': 'place_gender', 
                                'Name': 'full_name', 
                                'Runner no':'bib_number',
                                'Category': 'age_class', 
                                'Half': 'half_split', 
                                'Finish':'finish_time', 
                                }, inplace = True)

london_2017_df.to_csv( "Data/london/London_race_results_2017.csv")

#2018

london_2018_df.rename(columns = {
                                'Place overall': 'place_overall', 
                                'Place gender': 'place_gender', 
                                'Name': 'full_name', 
                                'Runner no':'bib_number',
                                'Category': 'age_class', 
                                'Half': 'half_split', 
                                'Finish':'finish_time', 
                                }, inplace = True)

london_2018_df.to_csv( "Data/london/London_race_results_2018.csv")

In [65]:
data_dir = 'Data/london/'

extension = 'csv'
all_filenames = [i for i in glob.glob(data_dir + '*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
if not os.path.exists(data_dir + 'output'):
    os.makedirs(data_dir + 'output')
combined_csv.to_csv(data_dir + "output/combined_london.csv", index=False, encoding='utf-8-sig')

/Users/bethmcgregor/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/bethmcgregor/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (8,9,11,17) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [66]:
combined_csv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 232623 entries, 0 to 37340
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                232623 non-null  int64  
 1   place_overall             232623 non-null  object 
 2   place_gender              232623 non-null  object 
 3   full_name                 232623 non-null  object 
 4   bib_number                232623 non-null  object 
 5   age_class                 232578 non-null  object 
 6   half_split                232149 non-null  object 
 7   finish_time               232623 non-null  object 
 8   gender                    232623 non-null  object 
 9   year                      232623 non-null  int64  
 10  Unnamed: 0.1              190005 non-null  float64
 11  Unnamed: 0.1.1            190005 non-null  float64
 12  Unnamed: 0.1.1.1          190005 non-null  float64
 13  Unnamed: 0.1.1.1.1        190005 non-null  fl

In [67]:
# Load the combined csv file created above
london_results_df = pd.read_csv('Data/london/output/combined_london.csv',)
london_results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232623 entries, 0 to 232622
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                232623 non-null  int64  
 1   place_overall             232623 non-null  object 
 2   place_gender              232623 non-null  object 
 3   full_name                 232623 non-null  object 
 4   bib_number                232623 non-null  object 
 5   age_class                 232578 non-null  object 
 6   half_split                232149 non-null  object 
 7   finish_time               232623 non-null  object 
 8   gender                    232623 non-null  object 
 9   year                      232623 non-null  int64  
 10  Unnamed: 0.1              190005 non-null  float64
 11  Unnamed: 0.1.1            190005 non-null  float64
 12  Unnamed: 0.1.1.1          190005 non-null  float64
 13  Unnamed: 0.1.1.1.1        190005 non-null  f

/Users/bethmcgregor/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1,2,4,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


NOTE: The mixed data types warning from above will be addressed by dropping columns or modifying data types below. 

In [68]:
# There are a number of columns that are not required, drop these before adding addtional course and weather columns

london_results_df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 11','Unnamed: 0.1.1.1.1'], axis = 1, inplace = True)

In [69]:
# Verify that these columns were dropped
london_results_df.head()


,place_overall,place_gender,full_name,bib_number,age_class,half_split,finish_time,gender,year,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1
0,1,1,"Ghebresilasie, Weynay (ERI)",1451,18-39,01:05:04,02:17:21,M,2019,NaN,NaN,NaN
1,2,2,"Gilbert, John (GBR)",1291,18-39,01:08:00,02:19:03,M,2019,NaN,NaN,NaN
2,3,3,"Johnson, Ben (GBR)",1246,18-39,01:09:22,02:20:03,M,2019,NaN,NaN,NaN
3,4,4,"Milne, Alex (GBR)",1456,18-39,01:09:30,02:20:08,M,2019,NaN,NaN,NaN
4,5,5,"Ruddy, Craig (GBR)",1362,18-39,01:07:58,02:20:29,M,2019,NaN,NaN,NaN


#### Split the full name columns into first name, last name, and nationality

In [70]:
# Split the full_name column in to separate nationality, first name and last name columns
london_results_df['nationality'] = london_results_df['full_name'].str.extract(r'\(([A-Z]{3})\)$')
london_results_df['last_name'] = london_results_df['full_name'].str.extract(r'^(.+),')
london_results_df['first_name'] = london_results_df['full_name'].str.extract(r'^.+, (.+) \([A-Z]{3}\)$')


In [71]:

london_results_df['first_namex'] = london_results_df.loc[london_results_df['nationality'].isna(), "full_name"].str.extract(r'^.+, (.+)$')
london_results_df['first_name'] = london_results_df['first_name'].fillna(london_results_df['first_namex'])
london_results_df = london_results_df.drop('first_namex', axis=1)


In [72]:
london_results_df['nationality'] = london_results_df['nationality'].fillna('Unknown')

In [73]:
# Check for missing values after the above step
london_results_df.isna().sum()

place_overall                   0
place_gender                    0
full_name                       0
bib_number                      0
age_class                      45
half_split                    474
finish_time                     0
gender                          0
year                            0
Unnamed: 0.1.1.1.1.1        42618
Unnamed: 0.1.1.1.1.1.1      42618
Unnamed: 0.1.1.1.1.1.1.1    42618
nationality                     0
last_name                       3
first_name                      4
dtype: int64

In [74]:
# Combine first name, and last name to create a single full name column

london_results_df['full_name']= london_results_df['first_name'] + ' ' + london_results_df['last_name']
london_results_df.head()

,place_overall,place_gender,full_name,bib_number,age_class,half_split,finish_time,gender,year,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,nationality,last_name,first_name
0,1,1,Weynay Ghebresilasie,1451,18-39,01:05:04,02:17:21,M,2019,NaN,NaN,NaN,ERI,Ghebresilasie,Weynay
1,2,2,John Gilbert,1291,18-39,01:08:00,02:19:03,M,2019,NaN,NaN,NaN,GBR,Gilbert,John
2,3,3,Ben Johnson,1246,18-39,01:09:22,02:20:03,M,2019,NaN,NaN,NaN,GBR,Johnson,Ben
3,4,4,Alex Milne,1456,18-39,01:09:30,02:20:08,M,2019,NaN,NaN,NaN,GBR,Milne,Alex
4,5,5,Craig Ruddy,1362,18-39,01:07:58,02:20:29,M,2019,NaN,NaN,NaN,GBR,Ruddy,Craig


#### First, convert any dataypes that don't match with other race dataframes so that name values can be matched on place_overall and year

In [75]:
# Convert bib number and place overall to numeric, any nonnumeric values are convered to NA

london_results_df['place_overall'] = pd.to_numeric(london_results_df['place_overall'], errors='coerce')

# Remove any NA values
london_results_df = london_results_df[~london_results_df['place_overall'].isna()]

# Convert place overall to integers

london_results_df['place_overall'] = london_results_df['place_overall'].astype(int)

                                                                              

In [76]:
london_results_df[london_results_df.place_overall.isna()]

,place_overall,place_gender,full_name,bib_number,age_class,half_split,finish_time,gender,year,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,nationality,last_name,first_name


In [77]:
london_results_df.dtypes

place_overall                 int64
place_gender                 object
full_name                    object
bib_number                   object
age_class                    object
half_split                   object
finish_time                  object
gender                       object
year                          int64
Unnamed: 0.1.1.1.1.1        float64
Unnamed: 0.1.1.1.1.1.1      float64
Unnamed: 0.1.1.1.1.1.1.1    float64
nationality                  object
last_name                    object
first_name                   object
dtype: object

In [78]:
# Splitting the previous names column from nationality resulted in 4 missing values in the full_name column
london_results_df[london_results_df['full_name'].isna()]

,place_overall,place_gender,full_name,bib_number,age_class,half_split,finish_time,gender,year,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,nationality,last_name,first_name
6749,8616,6750,NaN,40202,18-39,01:54:05,03:38:17,M,2019,NaN,NaN,NaN,Unknown,Mackenzie,NaN
49224,8343,6607,NaN,59130,18-39,01:51:25,03:54:00,M,2018,6606.0,6606.0,606.0,ITA,NaN,NaN
138444,26195,18060,NaN,50689,45-49,02:25:55,05:02:40,M,2014,18059.0,18059.0,59.0,GBR,NaN,NaN
152033,28075,9041,NaN,37050,18-39,02:20:46,05:13:59,W,2014,31648.0,31648.0,40.0,GBR,NaN,NaN


It seems these missing values were due to either missing first names or last names on the race results page
Since names will likely not be used for analysis purposes and serve mainly as a way to id runners, I've just included whatever name was provided in the record. 

The Missing Names: 
- 28075, 2014 = 'Harriet'
- 26195, 2014 = 'Michael James'
- 8343, 2018 = 'Andrea'
- 8616, 2019 = 'Mackenzie'



In [79]:
# Replace the names found above into the correct rows:

london_results_df.loc[(london_results_df.place_overall == 28075) & (london_results_df.year == 2014),'full_name']='Harriet'
london_results_df.loc[(london_results_df.place_overall == 26195) & (london_results_df.year == 2014),'full_name']='Michael James'
london_results_df.loc[(london_results_df.place_overall == 8343) & (london_results_df.year == 2018),'full_name']='Andrea'
london_results_df.loc[(london_results_df.place_overall == 8616) & (london_results_df.year == 2019),'full_name']='Mackenzie'


In [80]:
# Check again for missing values: 
london_results_df.isna().sum()

place_overall                   0
place_gender                    0
full_name                       0
bib_number                      0
age_class                      45
half_split                    474
finish_time                     0
gender                          0
year                            0
Unnamed: 0.1.1.1.1.1        42618
Unnamed: 0.1.1.1.1.1.1      42618
Unnamed: 0.1.1.1.1.1.1.1    42618
nationality                     0
last_name                       3
first_name                      4
dtype: int64

#### Investigate the missing values in the half_split column
There are 474 NaN values + ~150 values with the text '\<span class="text-muted">–\</span>\'. For whatever reason, this appears to be an error with the data recording at the time of the race. There are finish times for all of these athletes. As I would like to try using half_split as a predictor for finish time, it does not make sense to impute the missing half_split times by taking half of their finish time. The rows with missing half_split times will be dropped. 

In [81]:
london_results_df[london_results_df['half_split'].isna()]

,place_overall,place_gender,full_name,bib_number,age_class,half_split,finish_time,gender,year,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,nationality,last_name,first_name
42816,199,199,Matthew Coakes,1373,40-44,NaN,02:42:14,M,2018,198.0,198.0,198.0,GBR,Coakes,Matthew
43200,592,583,Jamie Stephenson,20640,18-39,NaN,02:53:26,M,2018,582.0,582.0,582.0,GBR,Stephenson,Jamie
43341,737,724,Simon Mead,1583,45-49,NaN,02:55:57,M,2018,723.0,723.0,723.0,GBR,Mead,Simon
45165,2738,2548,Dean Clapham,26940,50-54,NaN,03:18:04,M,2018,2547.0,2547.0,547.0,GBR,Clapham,Dean
45516,3174,2899,Daniel Allaway,32849,50-54,NaN,03:21:47,M,2018,2898.0,2898.0,898.0,GBR,Allaway,Daniel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232555,39269,15454,Janet Foulds,39174,65-69,NaN,08:27:48,W,2017,37273.0,37273.0,453.0,GBR,Foulds,Janet
232556,39271,15455,Angela Clegg,39967,45-49,NaN,08:29:08,W,2017,37274.0,37274.0,454.0,GBR,Clegg,Angela
232558,39276,15457,Suzie Featherstone,39470,45-49,NaN,08:35:02,W,2017,37276.0,37276.0,456.0,GBR,Featherstone,Suzie
232562,39280,15461,Georgina Elizabeth Feasey,8331,18-39,NaN,08:50:19,W,2017,37280.0,37280.0,460.0,GBR,Feasey,Georgina Elizabeth


In [82]:
# Drop the missing age_class values (NaN)
london_results_df = london_results_df[~london_results_df['half_split'].isna()]

In [83]:
# Drrop the missing age_class values ('<span class="text-muted">–</span>')
london_results_df = london_results_df[london_results_df['half_split']!='<span class="text-muted">–</span>']

#### Investigate the missing values in the age_class column

In [84]:
london_results_df[london_results_df['age_class'].isna()]

,place_overall,place_gender,full_name,bib_number,age_class,half_split,finish_time,gender,year,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,nationality,last_name,first_name
42661,44,44,Austin Roth,1992,NaN,01:09:39,02:32:34,M,2018,43.0,43.0,43.0,USA,Roth,Austin
42690,73,73,Jason Simpson,1993,NaN,01:12:36,02:35:24,M,2018,72.0,72.0,72.0,USA,Simpson,Jason
42903,287,286,Chikara Yanada,1996,NaN,01:21:23,02:45:47,M,2018,285.0,285.0,285.0,JPN,Yanada,Chikara
45001,2548,2384,Alexandru Rafa,13332,NaN,01:28:55,03:16:08,M,2018,2383.0,2383.0,383.0,GBR,Rafa,Alexandru
46627,4548,4010,Yi Jiang,51215,NaN,01:37:57,03:31:23,M,2018,4009.0,4009.0,9.0,CHN,Jiang,Yi
49056,8081,6439,Matthew Walkey,20763,NaN,01:40:55,03:52:40,M,2018,6438.0,6438.0,438.0,GBR,Walkey,Matthew
49737,9151,7120,John McEwing,50146,NaN,01:55:50,03:57:36,M,2018,7119.0,7119.0,119.0,GBR,McEwing,John
58379,22862,15763,Amandeep Sangha,3558,NaN,02:11:28,04:59:15,M,2018,15761.0,15761.0,761.0,GBR,Sangha,Amandeep
66296,271,1,Georgia Porter,970,NaN,01:20:15,02:44:50,W,2018,23678.0,23678.0,0.0,USA,Porter,Georgia
66777,4340,482,Yvonne Mery,61672,NaN,01:43:06,03:29:54,W,2018,24159.0,24159.0,481.0,CAN,Mery,Yvonne


There are 45 missing values. There doesn't appear to be a straightforward way to impute missing values. For the time being, these rows will be dropped.

In [85]:
# Drop the missing age_class values
london_results_df = london_results_df[~london_results_df['age_class'].isna()]

In [86]:
# Check for missing values
london_results_df.isna().sum()

place_overall                   0
place_gender                    0
full_name                       0
bib_number                      0
age_class                       0
half_split                      0
finish_time                     0
gender                          0
year                            0
Unnamed: 0.1.1.1.1.1        42484
Unnamed: 0.1.1.1.1.1.1      42484
Unnamed: 0.1.1.1.1.1.1.1    42484
nationality                     0
last_name                       3
first_name                      4
dtype: int64

#### Add columns with the course summary data to the dataframe

In [87]:
london_results_df['race'] = 'london'
london_results_df['altitude_start'] = 39.75
london_results_df['altitude_finish'] = 6.95
london_results_df['max_slope'] = 5.50
london_results_df['min_slope'] = -10.30
london_results_df['elevation_gain'] = 137.40
london_results_df['elevation_loss'] = -170.20
london_results_df['course_turns'] = 56

#### Add the weather data for each year

In [88]:
# First, create a dictionary with the min and max temperatures and precipitation for each year

london_weather_dictionary = {}
london_weather_dictionary[2014] = {
    'temperature_min': 7.9,
    'temperature_max': 15.8, 
    'precipitation': 0.00
} 

london_weather_dictionary[2015] = {
    'temperature_min': 7.2,
    'temperature_max': 13.4, 
    'precipitation': 0.00
} 


london_weather_dictionary[2016] = {
    'temperature_min': 3.1,
    'temperature_max': 12.1, 
    'precipitation': 0.00
} 

london_weather_dictionary[2017] = {
    'temperature_min': 7.1,
    'temperature_max': 13.6, 
    'precipitation': 0.00
} 

london_weather_dictionary[2018] = {
    'temperature_min': 12.0,
    'temperature_max': 16.7, 
    'precipitation': 0.00
} 

london_weather_dictionary[2019] = {
    'temperature_min': 7.1,
    'temperature_max': 16.1, 
    'precipitation': 0.00
} 

# Loop through the dictionary to add the min and max temperatures, and precipitation data for each race year

for y, v in london_weather_dictionary.items():
    london_results_df.loc[london_results_df['year'] == y, 'temperature_min'] = v['temperature_min']
    london_results_df.loc[london_results_df['year'] == y, 'temperature_max'] = v['temperature_max']
    london_results_df.loc[london_results_df['year'] == y, 'precipitation'] = v['precipitation']

In [89]:
# Confirm the course summary and weather data was added

london_results_df.head()

,place_overall,place_gender,full_name,bib_number,age_class,half_split,finish_time,gender,year,Unnamed: 0.1.1.1.1.1,...,altitude_start,altitude_finish,max_slope,min_slope,elevation_gain,elevation_loss,course_turns,temperature_min,temperature_max,precipitation
0,1,1,Weynay Ghebresilasie,1451,18-39,01:05:04,02:17:21,M,2019,NaN,...,39.75,6.95,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0
1,2,2,John Gilbert,1291,18-39,01:08:00,02:19:03,M,2019,NaN,...,39.75,6.95,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0
2,3,3,Ben Johnson,1246,18-39,01:09:22,02:20:03,M,2019,NaN,...,39.75,6.95,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0
3,4,4,Alex Milne,1456,18-39,01:09:30,02:20:08,M,2019,NaN,...,39.75,6.95,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0
4,5,5,Craig Ruddy,1362,18-39,01:07:58,02:20:29,M,2019,NaN,...,39.75,6.95,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0


#### Locate and remove any DSQ results

The steps below remove any disqualified (DSQ) results from the dataframe. 

In [90]:
london_results_df = london_results_df[london_results_df['place_overall']!='DSQ']

In [91]:
# Confirm no DSQ values remain
london_results_df[london_results_df['place_overall']=='DSQ']

,place_overall,place_gender,full_name,bib_number,age_class,half_split,finish_time,gender,year,Unnamed: 0.1.1.1.1.1,...,altitude_start,altitude_finish,max_slope,min_slope,elevation_gain,elevation_loss,course_turns,temperature_min,temperature_max,precipitation


#### Convert finish time and half split to seconds

In [92]:
def convert_to_seconds(time):
    x = time.split(":")
    return int(x[0])*3600+int(x[1])*60+int(x[2])

london_results_df['finish_time_seconds'] = london_results_df['finish_time'].apply(convert_to_seconds)
london_results_df['half_split_seconds'] = london_results_df['half_split'].apply(convert_to_seconds)

In [93]:
london_results_df.head()

,place_overall,place_gender,full_name,bib_number,age_class,half_split,finish_time,gender,year,Unnamed: 0.1.1.1.1.1,...,max_slope,min_slope,elevation_gain,elevation_loss,course_turns,temperature_min,temperature_max,precipitation,finish_time_seconds,half_split_seconds
0,1,1,Weynay Ghebresilasie,1451,18-39,01:05:04,02:17:21,M,2019,NaN,...,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0,8241,3904
1,2,2,John Gilbert,1291,18-39,01:08:00,02:19:03,M,2019,NaN,...,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0,8343,4080
2,3,3,Ben Johnson,1246,18-39,01:09:22,02:20:03,M,2019,NaN,...,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0,8403,4162
3,4,4,Alex Milne,1456,18-39,01:09:30,02:20:08,M,2019,NaN,...,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0,8408,4170
4,5,5,Craig Ruddy,1362,18-39,01:07:58,02:20:29,M,2019,NaN,...,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0,8429,4078


#### Create an elite_status column to distinguish elite runners
This was done by creating a dictionary with lists of elite runners for each year. Rather then go back and re-do the webscraping, I decided to create a dictionary with lists of the elite runners for each year. The lists of elite athletes for each year were obtained directly from the London Marathon results page. 

In [94]:
elite_names = {
    2014: ['Wilson Kipsang','Stanley Biwott', 'Tsegaye Kebede', 'Ayele Abshero', 'Tsegaye Mekonnen', 'Geoffrey Mutai', 'Emmanuel Mutai', 'Mo Farah', 'Feyisa Lilesa', 'Ryan Vail', 'Chris Thompson', 'Stephen Kiprotich', 'Reid Coolsaet', 'Pedro Nimo', 'Ben Livesey', 'Samuel Tsegay', 'Scott Overall', 'Yared Hagos', 'Edna Kiplagat', 'Florence Kiplagat', 'Tirunesh Dibaba', 'Feyse Tadese', 'Aberu Kebede', 'Jessica Augusto', 'Ana Dulce Felix', 'Tiki Gelana', 'Yuko Shimizu', 'Diane Nukuri-Johnson', 'Amy Whitehead', 'Emma Stepto'],
    2015: ['Eliud Kipchoge', 'Wilson Kipsang', 'Dennis Kimetto', 'Stanley Biwott', 'Tilahun Regassa', 'Samuel Kitwara', 'Javier Guerra', 'Ghebre Kibrom', 'Aleksey Reunkov', 'Serhiy Lebid', 'Emmanuel Mutai', 'Michael Shelley', 'Scott Overall', 'Anuradha Cooray', 'Koen Raymaekers', 'Hermano Ferreira', 'Mathew Hynes', 'Bekir Karayel', 'Christian Kreienbuehl', 'Cesar Lizano', 'Stijn Fincioen', 'Tigist Tufa', 'Mary Keitany', 'Tirfi Tsegaye', 'Aselefech Mergia', 'Florence Kiplagat', 'Jemima Sumgong', 'Priscah Jeptoo', 'Ana Dulce Felix', 'Volha Mazuronak', 'Edna Kiplagat', 'Iwona Lewandowska', 'Diane Nukuri', 'Tatyana Arkhipova', 'Alessandra Aguilar', 'Sonia Samuels', 'Mary Davies', 'Emma Stepto', 'Rebecca Robinson'],
    2016: ['Eliud Kipchoge', 'Stanley Biwott', 'Kenenisa Bekele', 'Ghirmay Ghebreslassie', 'Wilson Kipsang', 'Tilahun Regassa', 'Sisay Lemma', 'Callum Hawkins', 'Dennis Kimetto', 'Ghebre Kibrom', 'Yuki Sato', 'Tsegai Tewelde', 'Vitaliy Shafar', 'Derek Hawkins', 'Serhiy Lebid', 'Chris Thompson', 'Mathew Bond', 'Robbie Simpson', 'Lee Merrien', 'Andrew Davies', 'Rob Watson', 'Ben Moreau', 'Antonio Uribe', 'Stephen Scullion', 'Abera Kuma', 'Jonathan Hay', 'Craig Mottram', 'Jemima Sumgong', 'Tigist Tufa', 'Florence Kiplagat', 'Volha Mazuronak', 'Aselefech Mergia', 'Mare Dibaba', 'Feyse Tadese', 'Priscah Jeptoo', 'Mary Keitany', 'Jessica Augusto', 'Katarzyna Kowalska', 'Sara Hall', 'Alyson Dixon', 'Sonia Samuels', 'Irvette Van Zyl', 'Charlotte Purdue', 'Cassie Fien', 'Freya Ross', 'Rene Kalmer'],
    2017: ['Daniel WANJIRU', 'Kenenisa BEKELE', 'Bedan KAROKI', 'Abel KIRUI', 'Alphonce SIMBU', 'Ghirmay GHEBRESLASSIE', 'Asefa MENGSTU', 'Amanuel MESEL', 'Javier GUERRA', 'Michael SHELLEY', 'Ayad LAMDASSEM', 'Feyisa LILESA', 'Ghebre KIBROM', 'Abdellatif MEFTAH', 'Robbie SIMPSON', 'Andrew DAVIES', 'Tesfaye ABERA', 'Jesus Arturo ESPARZA', 'Scott OVERALL', 'Kevin SEAWARD', 'Tasama MOOGAS', 'Mick CLOHISEY', 'Jonny MELLOR', 'Tilahun REGASSA', 'Tom ANDERSON', 'Matt BOND', 'Ian KIMPTON', 'Jonathan HAY', 'Andrew LEMONCELLO', 'Chris THOMPSON', 'Diego ELIZONDO', 'Mary KEITANY', 'Tirunesh DIBABA', 'Aselefech MERGIA', 'Vivian CHERUIYOT', 'Lisa WEIGHTMAN', 'Laura THWEATT', 'Helah KIPROP', 'Tigist TUFA', 'Florence KIPLAGAT', 'Jessica TRENGOVE', 'Aberu KEBEDE', 'Diana LOBACEVSKE', 'Kellyn TAYLOR', 'Alyson DIXON', 'Charlotte PURDUE', 'Tracy BARLOW', 'Andrea DEELSTRA', 'Tish JONES', 'Melanie PANAYIOTOU', 'Hanna VANDENBUSSCHE', 'Susan PARTRIDGE', 'Jenny SPINK', 'Casey WOOD', 'Laura Graham', 'Krista DUCHENE', 'Barbara SANCHEZ'],
    2018: ['Eliud KIPCHOGE', 'Tola Shura KITATA', 'Mo FARAH', 'Abel KIRUI', 'Bedan KAROKI', 'Kenenisa BEKELE', 'Lawrence CHERONO', 'Daniel WANJIRU', 'Amanuel MESEL', 'Yohanes GEBREGERGISH', 'Ihor OLEFIRENKO', 'Stephen SCULLION', 'Fernando CABADA', 'Jonny MELLOR', 'Samuel CHELANGA', 'Tatsunori HAMASAKI', 'Guye ADOLA', 'Matthew CLOWES', 'Vivian CHERUIYOT', 'Brigid KOSGEI', 'Tadelech BEKELE', 'Gladys CHERONO', 'Mary KEITANY', 'Rose CHELIMO', 'Mare DIBABA', 'Lily PARTRIDGE', 'Tracy BARLOW', 'Stephanie BRUCE', 'Rebecca WADE', 'Rebecca MURRAY', 'Liz COSTELLO'],
    2019: ['Eliud KIPCHOGE', 'Mosinet GEREMEW', 'Mule WASIHUN', 'Tola Shura KITATA', 'Mo FARAH', 'Tamirat TOLA', 'Bashir ABDI', 'Leul GEBRESILASIE', 'Yassine RACHIK', 'Callum HAWKINS', 'Daniel WANJIRU', 'Wilson KIPSANG', 'Brett ROBINSON', 'Jack RAYNER', 'Daniel Chaves DA SILVA', 'Dewi GRIFFITHS', 'Ihor OLEFIRENKO', 'Henryk SZOST', 'Jonathan MELLOR', 'Derlis AYALA', 'Joshua GRIFFITHS', 'Thomas DE BOCK', 'Robbie SIMPSON', 'Colin LEAK', 'Mick CLOHISEY', 'Andy DAVIES', 'Nitender Singh RAWAT', 'Nicholas TORRY', 'Tsegai TEWELDE', 'Paul MARTELLETTI', 'Gary O\'HANLON', 'Yohei SUZUKI', 'Sean HEHIR', 'Iraitz ARROSPIDE', 'Dennis LAERTE', 'David CRINITI', 'Brigid KOSGEI', 'Vivian CHERUIYOT', 'Roza DEREJE', 'Gladys CHERONO', 'Mary KEITANY', 'Emily SISSON', 'Sinead DIVER', 'Carla Salome ROCHA', 'Birhane DIBABA', 'Charlotte PURDUE', 'Linet MASAI', 'Molly HUDDLE', 'Yuka ANDO', 'Lilia FISIKOVICI', 'Mao ICHIYAMA', 'Tish JONES', 'Lily PARTRIDGE', 'Hayley CARRUTHERS', 'Tracy BARLOW', 'Sonia SAMUELS', 'Natasha COCKRAM']
}
elite_names

london_results_df['elite_status'] = 0

# The for loop includes a check to make sure that the athlete names in the dictionary above match the names in the results
for k, v in elite_names.items():
    print(k)
    print('Expected Length:', len(v))
    print('Actual Length:', len(london_results_df.loc[(london_results_df['year']==k) & (london_results_df['place_overall']<100) & (london_results_df['full_name'].isin(v))]))
    
    london_results_df.loc[(london_results_df['year']==k) & (london_results_df['place_overall']<100) & (london_results_df['full_name'].isin(v)), 'elite_status'] = 1
                                            
#london_results_df.loc[(london_results_df['year']==2014) & (london_results_df['full_name'].isin(elite_names[2014]))]

2014
Expected Length: 30
Actual Length: 30
2015
Expected Length: 39
Actual Length: 39
2016
Expected Length: 46
Actual Length: 46
2017
Expected Length: 57
Actual Length: 57
2018
Expected Length: 31
Actual Length: 31
2019
Expected Length: 57
Actual Length: 57


In [95]:
# Confirm presence of new elite_status column
london_results_df.head()

,place_overall,place_gender,full_name,bib_number,age_class,half_split,finish_time,gender,year,Unnamed: 0.1.1.1.1.1,...,min_slope,elevation_gain,elevation_loss,course_turns,temperature_min,temperature_max,precipitation,finish_time_seconds,half_split_seconds,elite_status
0,1,1,Weynay Ghebresilasie,1451,18-39,01:05:04,02:17:21,M,2019,NaN,...,-10.3,137.4,-170.2,56,7.1,16.1,0.0,8241,3904,0
1,2,2,John Gilbert,1291,18-39,01:08:00,02:19:03,M,2019,NaN,...,-10.3,137.4,-170.2,56,7.1,16.1,0.0,8343,4080,0
2,3,3,Ben Johnson,1246,18-39,01:09:22,02:20:03,M,2019,NaN,...,-10.3,137.4,-170.2,56,7.1,16.1,0.0,8403,4162,0
3,4,4,Alex Milne,1456,18-39,01:09:30,02:20:08,M,2019,NaN,...,-10.3,137.4,-170.2,56,7.1,16.1,0.0,8408,4170,0
4,5,5,Craig Ruddy,1362,18-39,01:07:58,02:20:29,M,2019,NaN,...,-10.3,137.4,-170.2,56,7.1,16.1,0.0,8429,4078,0


#### Drop columns that are no longer needed

In [96]:
london_results_df = london_results_df.drop(['place_gender', 'half_split', 'finish_time', 'last_name', 'first_name'], axis = 1)
london_results_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231962 entries, 0 to 232622
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   place_overall             231962 non-null  int64  
 1   full_name                 231962 non-null  object 
 2   bib_number                231962 non-null  object 
 3   age_class                 231962 non-null  object 
 4   gender                    231962 non-null  object 
 5   year                      231962 non-null  int64  
 6   Unnamed: 0.1.1.1.1.1      189478 non-null  float64
 7   Unnamed: 0.1.1.1.1.1.1    189478 non-null  float64
 8   Unnamed: 0.1.1.1.1.1.1.1  189478 non-null  float64
 9   nationality               231962 non-null  object 
 10  race                      231962 non-null  object 
 11  altitude_start            231962 non-null  float64
 12  altitude_finish           231962 non-null  float64
 13  max_slope                 231962 non-null  f

In [97]:
# Add a column with the race name to the dataframe

london_results_df['marathon_name'] = 'london'

# Check that `race` column was added
london_results_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231962 entries, 0 to 232622
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   place_overall             231962 non-null  int64  
 1   full_name                 231962 non-null  object 
 2   bib_number                231962 non-null  object 
 3   age_class                 231962 non-null  object 
 4   gender                    231962 non-null  object 
 5   year                      231962 non-null  int64  
 6   Unnamed: 0.1.1.1.1.1      189478 non-null  float64
 7   Unnamed: 0.1.1.1.1.1.1    189478 non-null  float64
 8   Unnamed: 0.1.1.1.1.1.1.1  189478 non-null  float64
 9   nationality               231962 non-null  object 
 10  race                      231962 non-null  object 
 11  altitude_start            231962 non-null  float64
 12  altitude_finish           231962 non-null  float64
 13  max_slope                 231962 non-null  f

In [98]:
# Check for duplicate rows
london_results_df.duplicated().sum()

0

#### Re-order columns to desired order

In [99]:
# Create a dataframe with columns common to Chicago and Berlin
london_results_reduced_df = london_results_df[['marathon_name', 'year', 'place_overall', 'full_name', 'elite_status', 'nationality', 'gender', 'finish_time_seconds', 'altitude_start', 'altitude_finish', 'max_slope', 'min_slope', 'elevation_gain', 'elevation_loss', 'course_turns', 'temperature_min', 'temperature_max', 'precipitation']]
london_results_reduced_df.head()

,marathon_name,year,place_overall,full_name,elite_status,nationality,gender,finish_time_seconds,altitude_start,altitude_finish,max_slope,min_slope,elevation_gain,elevation_loss,course_turns,temperature_min,temperature_max,precipitation
0,london,2019,1,Weynay Ghebresilasie,0,ERI,M,8241,39.75,6.95,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0
1,london,2019,2,John Gilbert,0,GBR,M,8343,39.75,6.95,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0
2,london,2019,3,Ben Johnson,0,GBR,M,8403,39.75,6.95,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0
3,london,2019,4,Alex Milne,0,GBR,M,8408,39.75,6.95,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0
4,london,2019,5,Craig Ruddy,0,GBR,M,8429,39.75,6.95,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0


In [100]:
london_results_reduced_df.shape

(231962, 18)

In [101]:
# Do a final check for missing values
london_results_reduced_df.isna().sum()

marathon_name          0
year                   0
place_overall          0
full_name              0
elite_status           0
nationality            0
gender                 0
finish_time_seconds    0
altitude_start         0
altitude_finish        0
max_slope              0
min_slope              0
elevation_gain         0
elevation_loss         0
course_turns           0
temperature_min        0
temperature_max        0
precipitation          0
dtype: int64

In [102]:
# Save this output to a csv
london_results_reduced_df.to_csv("Data/london/output/london_results_reduced_df.csv")

In [103]:
# Create a dataframe retaining as many features as possible for London
london_results_all_features_df = london_results_df[['marathon_name', 'year', 'place_overall', 'full_name', 'bib_number', 'age_class', 'elite_status', 'half_split_seconds', 'nationality', 'gender', 'finish_time_seconds', 'altitude_start', 'altitude_finish', 'max_slope', 'min_slope', 'elevation_gain', 'elevation_loss', 'course_turns', 'temperature_min', 'temperature_max', 'precipitation']]
london_results_all_features_df.head()

,marathon_name,year,place_overall,full_name,bib_number,age_class,elite_status,half_split_seconds,nationality,gender,...,altitude_start,altitude_finish,max_slope,min_slope,elevation_gain,elevation_loss,course_turns,temperature_min,temperature_max,precipitation
0,london,2019,1,Weynay Ghebresilasie,1451,18-39,0,3904,ERI,M,...,39.75,6.95,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0
1,london,2019,2,John Gilbert,1291,18-39,0,4080,GBR,M,...,39.75,6.95,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0
2,london,2019,3,Ben Johnson,1246,18-39,0,4162,GBR,M,...,39.75,6.95,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0
3,london,2019,4,Alex Milne,1456,18-39,0,4170,GBR,M,...,39.75,6.95,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0
4,london,2019,5,Craig Ruddy,1362,18-39,0,4078,GBR,M,...,39.75,6.95,5.5,-10.3,137.4,-170.2,56,7.1,16.1,0.0


In [104]:
london_results_all_features_df.shape

(231962, 21)

In [105]:
# Save london_results_all_features_df to a csv
london_results_all_features_df.to_csv("Data/london/output/london_results_all_features_df.csv")

This notebook combined the marathon results obtained from webscraping, cleaned and processed the results to create consistency between the three races, and added weather and course features. The output of this notebook was 6 csv files: 
- 3 csv files with a reduced set of common features across each marathon:
    - chicago_results_reduced_df.csv 
    - london_results_reduced_df.csv
    - berlin_results_reduced.csv
These files will be used for EDA and basic modelling. 
    
- 3 csv files with as many features as possible for each marathon:  
    - chicago_results_all_features.csv
    - london_results_all_features_df.csv
    - berlin_results_all_features.csv
These files will be used for more advanced modelling. 